# HDS5210-2022 Midterm

In the midterm, you're going to use all of the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged: "home" if they are below 30% risk and the recommended hospital if they are at or above 30%.

Each step in the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure that you're getting those steps correct.

The midterm is due at 11:59 PM CST on Monday, March 14th.

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should raise a ValueException that includes a message with the invalid input and which parameter that input was provided as.

NOTES:
1. In the final step there is a rule to convert form raw score to percentile.  In that conversion, 17-25 maps to 59-88% and ≥26 maps to >99%.  For our purposes, we want these to be specific % number outputs.  Use the following rule:
 * If score is between 17 and 25, percentile should be 0.59
 * If score is greater than or equal to 26, percentile should be 0.99


In [1]:
def priest(sex, age, respiratory_rate, o2_sat, heart_rate, bp_sys, temp_C, alertness, o2_inspired, performance_status):
    '''The 1st part of this function calculates the severity of COVID-19 in a person taking their vitals and few other parameters into 
    consideration.The algorithm used is provided in the MDCalc website. Then based on the score we can predict the probability of adverse
    effects they might face in the next 30 days.
    
    >>> priest('FEMALE', 40, 24, 0.96, 105, 115, 39.9, 'Alert', 'AIR', 'Unrestricted normal activity')
    0.09
    >>> priest('MaLe', 32, 20, 0.93, 100, 110, 32.2, 'CONFUSED OR NOT ALERT', 'SUPPLEMENTAL OXYGEN', 'LIMITED SELF-CARE')
    0.55
    >>> priest('FemalE', 22, 17, 0.98, 96, 103, 31.0, 'AleRT', 'AiR', 'BED/CHAIR bOuNd, no SELF-care')
    0.26
    >>> priest('MalE', 65, 24, 0.87, 90, 99, 36.5, 'ConFusEd oR nOt AleRt', 'air', 'limited strenuous activity, can do light activity')
    0.47
    >>> priest('male', 50, 20, 0.90, 87, 95, 35, 'alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
    0.59
    '''
    if sex.lower() not in ['male', 'female']:
        raise ValueError(f'Invalid input for Sex:{sex.lower()}')
    if sex.lower() == "male":
        sex = 1
    elif sex.lower() == "female":
        sex = 0
    if age >= 16 and age <= 49:
        age = 0
    elif age >= 50 and age <= 65:
        age = 2
    elif age >= 66 and age <= 80:
        age = 3
    elif age > 80:
        age = 4
    if respiratory_rate < 9:
        respiratory_rate = 3
    elif respiratory_rate >= 9 and respiratory_rate <= 11:
        respiratory_rate = 1
    elif respiratory_rate >= 12 and respiratory_rate <= 20:
        respiratory_rate = 0
    elif respiratory_rate >= 21 and respiratory_rate <= 24:
        respiratory_rate = 2
    elif respiratory_rate > 24:
        respiratory_rate = 3
    if o2_sat > 0.95:
        o2_sat = 0
    elif o2_sat >= 0.94 and o2_sat <= 0.95:
        o2_sat = 1
    elif o2_sat >= 0.92 and o2_sat <= 0.93:
        o2_sat = 2
    elif o2_sat < 0.92:
        o2_sat = 3
    if heart_rate < 41:
        heart_rate = 3
    elif heart_rate >= 41 and heart_rate <= 50:
        heart_rate = 1
    elif heart_rate >= 51 and heart_rate <= 90:
        heart_rate = 0
    elif heart_rate >= 91 and heart_rate <= 110:
        heart_rate = 1
    elif heart_rate >= 111 and heart_rate <= 130:
        heart_rate = 2
    elif heart_rate >130:
        heart_rate = 3
    if bp_sys < 91:
        bp_sys = 3
    elif bp_sys >= 91 and bp_sys <= 100:
        bp_sys = 2
    elif bp_sys >= 101 and bp_sys <= 110:
        bp_sys = 1
    elif bp_sys >= 111 and bp_sys <= 219:
        bp_sys = 0
    elif bp_sys > 219:
        bp_sys = 3
    if temp_C < 35.1:
        temp_C = 3
    elif temp_C >= 35.1 and temp_C <= 36.0:
        temp_C = 1
    elif temp_C >= 36.1 and temp_C <= 38.0:
        temp_C = 0
    elif temp_C >= 38.1 and temp_C <= 39.0:
        temp_C = 1
    elif temp_C > 39.0:
        temp_C = 2
    if alertness.lower() == 'alert':
        alertness = 0
    elif alertness.lower() == 'confused or not alert':
        alertness = 3
    if o2_inspired.lower() == 'air':
        o2_inspired = 0
    elif o2_inspired.lower() == 'supplemental oxygen':
        o2_inspired = 2
    if performance_status.lower() == 'unrestricted normal activity':
        performance_status = 0
    elif performance_status.lower() == 'limited strenuous activity, can do light activity':
        performance_status = 1
    elif performance_status.lower() == 'limited activity, can self-care':
        performance_status = 2
    elif performance_status.lower() =='limited self-care':
        performance_status = 3
    elif performance_status.lower() == 'bed/chair bound, no self-care':
        performance_status = 4
    risk_score = sum([sex, age, respiratory_rate, o2_sat, heart_rate, bp_sys, temp_C, alertness, o2_inspired, performance_status])
    
    if risk_score >= 0 and risk_score <= 1:
        risk = 0.01
    elif risk_score >= 2 and risk_score <= 3:
        risk = 0.02
    elif risk_score == 4:
        risk = 0.03
    elif risk_score == 5:
        risk = 0.09
    elif risk_score == 6:
        risk = 0.15
    elif risk_score == 7:
        risk = 0.18
    elif risk_score == 8:
        risk = 0.22
    elif risk_score == 9:
        risk = 0.26
    elif risk_score == 10:
        risk = 0.29
    elif risk_score == 11:
        risk = 0.34
    elif risk_score == 12:
        risk = 0.38
    elif risk_score == 13:
        risk = 0.46
    elif risk_score == 14:
        risk = 0.47
    elif risk_score == 15:
        risk = 0.49
    elif risk_score == 16:
        risk = 0.55
    elif risk_score >= 17 and risk_score <= 25:
        risk = 0.59
    elif risk_score >= 26:
        risk = 0.99
    return risk

In [2]:
import doctest
doctest.run_docstring_examples(priest, globals(), verbose=True)

Finding tests in NoName
Trying:
    priest('FEMALE', 40, 24, 0.96, 105, 115, 39.9, 'Alert', 'AIR', 'Unrestricted normal activity')
Expecting:
    0.09
ok
Trying:
    priest('MaLe', 32, 20, 0.93, 100, 110, 32.2, 'CONFUSED OR NOT ALERT', 'SUPPLEMENTAL OXYGEN', 'LIMITED SELF-CARE')
Expecting:
    0.55
ok
Trying:
    priest('FemalE', 22, 17, 0.98, 96, 103, 31.0, 'AleRT', 'AiR', 'BED/CHAIR bOuNd, no SELF-care')
Expecting:
    0.26
ok
Trying:
    priest('MalE', 65, 24, 0.87, 90, 99, 36.5, 'ConFusEd oR nOt AleRt', 'air', 'limited strenuous activity, can do light activity')
Expecting:
    0.47
ok
Trying:
    priest('male', 50, 20, 0.90, 87, 95, 35, 'alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
Expecting:
    0.59
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk percetage of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None without raising any exceptions
5. Include a good docstring with at least five test cases.


In [3]:
import requests
import json
def find_hospital(age, sex, risk):
    '''This function takes the risk score, age and sex of a patient and figure out which is the nearest facility where the person can be taken
    if they are in some emergency using the REST web service.
    
    >>> find_hospital(45, 'male', 0.2)
    'Southwest Hospital and Medical Center'
    >>> find_hospital(65, 'male', 0.49)
    'Wesley Woods Geriatric Hospital'
    >>> find_hospital(29, 'female', 0.56)
    'Emory Dunwoody Medical Center'
    >>> find_hospital(19, 'female', 0.16)
    'Select Specialty Hospital - Northeast Atlanta'
    >>> find_hospital(22, 'female', 0.92)
    'Emory Dunwoody Medical Center'
    '''
    url = f'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}'
    response = requests.get(url)
    hospital_name = response.json()['hospital']
    return (hospital_name)

In [4]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(), verbose=True)

Finding tests in NoName
Trying:
    find_hospital(45, 'male', 0.2)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(65, 'male', 0.49)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(29, 'female', 0.56)
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospital(19, 'female', 0.16)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(22, 'female', 0.92)
Expecting:
    'Emory Dunwoody Medical Center'
ok


## Part 3: Get the address for that hospital from a webpage

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list on this webpage to lookup the address for that hospital, based on its name.

https://www.officialusa.com/stateguides/health/hospitals/georgia.html

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data on the webpage above to find the addres for that hospital.
2. I've said that all the hospitals are in Atlanta, but this webpage has hospitals from all of Georgia.  So, make sure you verify that the row of data you're using is in Atlanta, just in case there are hospitals with the same name in different cities.
3. Your code will have to parse the HTML on the webpage and turn that into some kind of data structure that you can search through to find the right hospital.
4. If you do find more than one matching hospital in Atlanta with the same name, you should raise a KeyError.
5. If the hospital name isn't found, the function should raise a KeyError.
6. Be sure to use good docstring documentation and includes at least 3 test cases.

In [5]:
from bs4 import BeautifulSoup
import requests

# Note that you'll need to fetch the data using the following syntax to include headers
url = 'https://www.officialusa.com/stateguides/health/hospitals/georgia.html'

headers = { "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36" }

response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.text, 'html.parser')

In [6]:
response.status_code

200

In [7]:
def get_address(hospital):
    ''' As we found out which hospital to approach in an emergency we should also find out the address of that facility. Now this function
    prints out the address of that facility from the website that has the list of hospitals and their address.
    
    >>> get_address('Select Specialty Hospital - Northeast Atlanta')
    '1821 CLIFTON ROAD NE'
    >>> get_address('Emory Dunwoody Medical Center')
    '4500 NORTH SHALLOWFORD ROAD'
    >>> get_address('Wesley Woods Geriatric Hospital')
    '1821 CLIFTON ROAD, NE'
    '''
    results = []
    for address_list in soup.find_all('tr'):
        for address in address_list.find_all('td'):
            if address.text == 'ATLANTA':
                next_siblings = address.find_next_siblings('td')
                if next_siblings[0].text.lower() == hospital.lower():
                    results.append(next_siblings[2].text)           
    if len(results)> 1:
        raise KeyError('More Than one Hospital Found')
    else:
        return results[0]

In [8]:
import doctest
doctest.run_docstring_examples(get_address, globals(), verbose=True)

Finding tests in NoName
Trying:
    get_address('Select Specialty Hospital - Northeast Atlanta')
Expecting:
    '1821 CLIFTON ROAD NE'
ok
Trying:
    get_address('Emory Dunwoody Medical Center')
Expecting:
    '4500 NORTH SHALLOWFORD ROAD'
ok
Trying:
    get_address('Wesley Woods Geriatric Hospital')
Expecting:
    '1821 CLIFTON ROAD, NE'
ok


## Part 4: Run the risk calculator on a population

The `/data` folder has a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.

In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes a file name as a parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you below.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [9]:
def process_people(file_name):
    results = {}
    with open("../data/"+file_name) as csv:
        peoples_data = csv.readlines()
        for people in peoples_data[1:]:
            try:
                patient_details = people.split("|")
                patient_number = patient_details[0]
                
#we are calling priest function and passing the data from patient details in the same order required for function.

                risk_score = priest(patient_details[1], int(patient_details[2]), int(patient_details[3]), float(patient_details[4]), 
                                     int(patient_details[5]), int(patient_details[6]),float(patient_details[7]), patient_details[8],
                                     patient_details[9],(patient_details[10][0:-1]))
    
# we are finding the hospital name by passing Age, Sec and the risk score which was calculated from the above priest function

                hospital_name = find_hospital(int(patient_details[2]), patient_details[1], risk_score)
                hospital_adddress = get_address(hospital_name)
                results[patient_number] = [patient_details[1], int(patient_details[2]), int(patient_details[3]), float(patient_details[4]), 
                     int(patient_details[5]), int(patient_details[6]),float(patient_details[7]), patient_details[8],
                     patient_details[9],(patient_details[10][0:-1]),risk_score, hospital_name, hospital_adddress]
            except Exception as e:
                print(e, people, hospital_name)
    return results

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` in the `/data` folder.  Write some code to check your results.  This does not need to be a function.

In [10]:
import json
json_file = "people_results.json"
with open("../data/"+json_file) as file:
    json_data = json.loads(file.read())
    peoples_results = process_people('people.psv')
    if json_data == peoples_results:
        print("Both are Same")

Both are Same


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [ ]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git add "midterm-2022.ipynb"
    !git commit -a -m "Submitting the midterm"
    !git push
else:
    print('''
    
OK. We can wait.
''')